In [ ]:
import torch # Importa biblioteca do pytorch
!python --version
from IPython.display import Image # Importa biblioteca para printar imagens

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # importa arquivos do google drive para o disco da máquina virtual

In [ ]:
!mkdir imagens
!unzip [CAMINHO DATASET UFPR] -d /content/imagens/ # descompacta imagens a serem utilizadas no treinamento/teste/validação
!git clone https://github.com/ultralytics/yolov5 # importa yolov5

In [ ]:
%cd yolov5
!pip install -r requirements.txt # instala bibliotecas necessárias para utilizar o yolov5
!pip install IProgress

# Treinamento do Modelo YoloV5

 > Copiar arquivos para treinamento.

 > Copiar arquivo dataset.yaml para o Diretório: yolov5/data.

In [ ]:
!python train.py --img 1056 --batch 8 --epochs 10 --data dataset.yaml --weights yolov5n.pt --cache

> Copia resultados do treinamento para pasta no google drive

In [ ]:
!cp -r /content/yolov5/runs/train/exp2/ -d [CAMINHO PARA SALVAR RESULTADOS]

# Testando o Modelo Treinado:

In [ ]:
!python detect.py --weights [MODELO TREINADO] --img 1056 --conf 0.25 --source [CAMINHO ARQUIVOS DE TESTES]

In [ ]:
# COPIA RESULTADOS
!cp -r /content/yolov5/runs/detect/exp/ -d [CAMINHO PARA SALVAR RESULTADOS]

> Testando modelo treinado de modo a obter as posições das bounding boxes e o grau de confiança de cada uma.

In [ ]:
dir = '/content/imagens/'
filename = '[CAMINHO]/filename_detecPlaca.txt' # ARQUIVO CONTENDO NOMES DAS IMAGENS DE TESTES
temp = open(filename, 'r').read().splitlines()

im = [dir + x for x in temp]

In [ ]:
# TESTES

model = torch.hub.load('ultralytics/yolov5', 'custom', '[MODELO TREINADO]')

In [ ]:
!cd images_inference

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
print('Resultados da Inferência no Conjunto de Testes:\n')

for i in range(0, len(im)):
 results = model(im[i])
 print(im[i][17:37])
 if results.xyxy[0].numel() == 0:
   continue
 tensor = results.xyxy[0][0]
 
 xi = tensor[0]
 xf = tensor[2]
 yi = tensor[1]
 yf = tensor[3]
 cf = tensor[4]
 xi = int(round(xi.item(), 0))
 xf = int(round(xf.item(), 0))
 yi = int(round(yi.item(), 0))
 yf = int(round(yf.item(), 0))
 bboxes = [i+1, yi, yf, xi, xf, cf.item()]
 print(bboxes)

 img = cv2.imread(im[i])
 img_crop = img[yi:yf, xi:xf]
 cv2_imshow(img_crop)


In [ ]:
import cv2

for i in range(0, len(im)):
 results = model(im[i])
 
 if results.xyxy[0].numel() == 0:
   continue
 tensor = results.xyxy[0][0]
 xi = tensor[0]
 xf = tensor[2]
 yi = tensor[1]
 yf = tensor[3]
 cf = tensor[4]
 cf = cf.item()
 cf = str(cf)
 cf = cf[0:5]
 cf = float(cf)
 #print(cf)
 xi = int(round(xi.item(), 0))
 xf = int(round(xf.item(), 0))
 yi = int(round(yi.item(), 0))
 yf = int(round(yf.item(), 0))
 bboxes = [i+1, yi, yf, xi, xf, cf]

 #print(im[i])
 img = cv2.imread(im[i])
 img_crop = img[yi:yf, xi:xf]
 #cv2.imwrite('/content/yolov5/images_inference/'+str(i+1)+'.png', img_crop)
 cv2.imwrite('/content/yolov5/images_inference/'+im[i][24:37]+'.png', img_crop)
 #cv2_imshow(img_crop)
 with open('bboxes.txt', 'a') as f:
    f.write('\n')
    f.writelines(str(bboxes))
 f.close()

In [ ]:
!cp -r /content/yolov5/images_inference2/ -d [CAMINHO PARA SALVAR RESULTADOS]

# Convertendo do formato pytorch para o formato TFLITE para poder utilizar em smartphones

In [ ]:
# Converte para formato fp16 TFlite

!python export.py --weights [MODELO TREINADO] --include tflite --data dataset.yaml --img 1056

In [ ]:
# Testando Modelo FP16

!python detect.py --weights /content/yolov5/runs/train/exp/weights/best-int8.tflite --img 1056 --conf 0.25 --data dataset.yaml --source [IMAGENS DE TESTE]

In [ ]:
!cp -r /content/yolov5/runs/detect/exp2/ -d /content/drive/MyDrive/ # COPIA ARQUIVOS DO MODELO CONVERTIDO

# Converte para formato int8

In [ ]:
!python export.py --weights [MODELO TREINADO] --include tflite --int8 --img 1056 --data  dataset.yaml